In [7]:
import requests
import tensorflow as tf
import numpy as np
import json
import matplotlib.pyplot as plt
import pandas as pd
import base64

In [4]:
hate = pd.read_csv("data/hate.csv")
hate

,text,recalled
0,A few months ago I was in the tube sitting dow...,1
1,A man on the phone was rude to me when discuss...,1
2,A nightmare neighbour of mine used to repeated...,1
3,A stranger is calling me a bitch all of a sudd...,0
4,As a retired person i have plenty of time to b...,1
...,...,...
499,One of my best friends is extremely jealous of...,1
500,A work colleague of mine looks at me and tells...,1
501,Late one evening I give a lift to two of my fr...,1
502,A friend of mine's boyfriend once called me an...,1


In [9]:
text_test = hate['text'][3]

text_test

'A stranger is calling me a bitch all of a sudden.I am doing my own thing whilst sitting at a bus stop.I feel awkward and upset for being called that.The stranger is sitting next to me looks at me in a hateful way.I get up and move to the end of the bus stop standing next to another person.I can still see the stranger looking at me which makes me feel awkward.I take my phone out of my bag and the stranger looks away.I think she wants attention and conflict.She wants me to react to her rudeness but i am not going to.'

In [11]:
def prepare_input_json(text_input):
    # Membuat spesifikasi fitur untuk input teks
    feature_specification = {
        "text": tf.train.Feature(bytes_list=tf.train.BytesList(value=[text_input.encode("utf-8")]))
    }
    
    # Membuat contoh TensorFlow dari spesifikasi fitur
    serialized_example = tf.train.Example(
        features=tf.train.Features(feature=feature_specification)
    ).SerializeToString()
    
    # Mengonversi contoh yang diserialisasi ke format base64
    encoded_example = base64.b64encode(serialized_example).decode()
    
    # Mengemas contoh yang diserialisasi dalam format JSON
    request_payload = [
        {
            "examples": {
                "b64": encoded_example
            }
        }
    ]
    
    # Mengembalikan string JSON untuk digunakan dalam permintaan prediksi
    return json.dumps({
        "signature_name": "serving_default",
        "instances": request_payload
    })

# Contoh teks untuk pengujian
test_text = "Text to be predict"
json_request = prepare_input_json(test_text)
    
# Endpoint untuk model TensorFlow Serving
model_endpoint = "http://localhost:8080/v1/models/hate-prediction-model:predict"
# Mengirim permintaan POST ke endpoint model dengan data JSON
server_response = requests.post(model_endpoint, data=json_request)

# Mendapatkan prediksi dari respon server
predictions = server_response.json().get("predictions")
if predictions:
    predicted_value = predictions[0][0]
    # Menentukan hasil berdasarkan nilai prediksi
    prediction_result = "Recalled" if prediction_value > 0.6 else "Imagined"
else:
    prediction_result = "Error: No predictions found in response."

print(prediction_result)


Imagined


Melakukan prediksi, jika hasil prediction value lebih dari 0.6 maka tergolong diingat, namun jika di bawah 0.6 maka tergolong tidak diingat